In [2]:
# data prepare
import glob,re,os,sys
os.chdir('C:/kongjy_special/实习工作内容总结/201805-201808华泰证券/Task4/')
sys.path.append('C:\\kongjy_special\\实习工作内容总结\\201805-201808华泰证券\\Task4')
from matplotlib import pyplot as plt
import pandas as pd
import time
import TranFeature,Tools_kjy
import warnings

warnings.filterwarnings('ignore')
pd.options.display.width = 200

# 文件名
TickNameList = glob.glob('Java_log_orders_600519/Tick*.csv')
TranNameList = glob.glob('Java_log_orders_600519/Tran*.csv')
# TestName是 开平仓点 的csv，需要和Tick以及Tran保持时间对齐
DateList = [re.search('\d{8}',i)[0] for i in TickNameList if re.search('\d{8}',i)]  
TestNameFun = lambda x: "Java_log_orders_600519\\600519.SH_back_test_info_" + x +".csv"
TestNameList = [TestNameFun(i) for i in DateList]
print (TickNameList[0])
print (TranNameList[0])
print (TestNameList[0])

Java_log_orders_600519\Tick20180502.csv
Java_log_orders_600519\Tran20180502.csv
Java_log_orders_600519\600519.SH_back_test_info_20180502.csv


In [3]:
# 所有日期的下单点的准确率
predict_label = 'Price'     # 待预测的量
DataSave = pd.DataFrame()   # 保存历史开仓点
Tick0 = pd.DataFrame()      # TickData 保存
# 数据读入 和 预处理
for date_index in range(len(TickNameList))[:3]:
    date = re.search('\d{8}',TickNameList[date_index])[0]
    tempTick = pd.read_csv(TickNameList[date_index])
    tempTran = pd.read_csv(TranNameList[date_index])
    tempTest = pd.read_csv(TestNameList[date_index])
    # Tick数据处理
    tempTick = tempTick.rename(columns={"Unnamed: 0":"Timestamp"})
    tempTick['Timestamp'] = pd.to_datetime(tempTick['Timestamp'])
    tempTick['median_5'] = (tempTick[predict_label].rolling(window=5).median()).shift(-2).fillna(method='pad')
    tempTick['median_5'],_ = Tools_kjy.smooth(tempTick['median_5'])
    # 开平仓点提取
    tickindex = tempTest[tempTest.order != " "].index
    print ("Date : %s"%(date))
    # # Tran 数据处理
    tempTran = tempTran.rename(columns={"Unnamed: 0":"Timestamp"})
    tempTran['Timestamp'] = pd.to_datetime(tempTran['Timestamp'])
    FeatureTran = TranFeature.TranFeature(tempTran,tempTick)
    tempTick = FeatureTran._Tran_features()

    # past change
    def past_change(x):
        tempTick['Past_diff'+str(x)] = tempTick[predict_label].diff(x).fillna(0)
    past_change(1);past_change(2);past_change(3);past_change(4);past_change(5);past_change(6)
    
    # Past_shift1
    tempTick['Past_shift1'] = tempTick[predict_label].shift(1)
    
    # future change -- 按照百分之表示
    tempTick['diff_3Tick'] = tempTick[predict_label].diff(3).shift(-3).fillna(0) / tempTick['Past_shift1'] * 100
    tempTick['diff_1Tick'] = tempTick[predict_label].diff(1).shift(-1).fillna(0) / tempTick['Past_shift1'] * 100
    tempTick['isadd_3Tick'] = tempTick['diff_3Tick'].apply(Tools_kjy.trinary)
    tempTick['isadd_1Tick'] = tempTick['diff_1Tick'].apply(Tools_kjy.trinary)
    
    # 等Volume间隔下的Price
    Tools_kjy.cumsum(tempTick)
    Tools_kjy.findequal(tempTick,5)
    Tools_kjy.findequal(tempTick,10)
    Tools_kjy.findequal(tempTick,15)
    Tools_kjy.findequal(tempTick,20)
    Tools_kjy.findequal(tempTick,25)
    Tools_kjy.BidAskPower(tempTick,"Bid")
    Tools_kjy.BidAskPower(tempTick,"Ask")
    
    periods = 6
    Tools_kjy.BidAskTime(tempTick,"Bid",periods)
    Tools_kjy.BidAskTime(tempTick,"Ask",periods)
    tempTick['TRAN_P1_ori'] = (-tempTick['BidStrength'] + tempTick['AskStrength']) 
    tempTick['TRAN_P1'] = tempTick['TRAN_P1_ori'].apply(Tools_kjy.trinary)
    
    #########################
    # 三部分的power
    # 价格变动能量--PS = mean(P1,P2,P3)
    # 交易变化能量--TS = mean(TRAN,t?)
    # 买卖变化力度--MS = mean(CountBid * MedianBidP + CountAsk * MedianAskP)
    tempTick['MedianBidP'] = tempTick['MedianBid'] - tempTick['Price']
    tempTick['MedianAskP'] = tempTick['MedianAsk'] - tempTick['Price']
    tempTick['PS'] = tempTick[['Past_diff1','Past_diff2','Past_diff3']].apply(pd.np.mean,axis=1)
    tempTick['MS'] = tempTick['CountBid'] * tempTick['MedianBidP'] + tempTick['CountAsk'] * tempTick['MedianAskP']
    tempTick['TS'] = tempTick['TRAN_P1_ori']
    tempTick['CountPrice'] = tempTick['CountPrice'].fillna(0)
    #########################
    
    # 开平仓点保存
    DataSave = DataSave.append(tempTick.loc[tickindex,:])
    Tick0 = Tick0.append(tempTick.loc[1:4725,])

DataSave = DataSave.reset_index(drop=True)
Tick0 = Tick0.reset_index(drop=True)
print ("done!")

Date : 20180502
Date : 20180503
Date : 20180504
done!


In [4]:
Tick0[['CountBid','CountAsk','MedianBidP','MedianAskP','Price','AskP0','BidP0','Past_diff1','Past_diff2',
          'Past_diff3','TRAN_P1_ori','diff_3Tick','diff_1Tick']].head(60)

,CountBid,CountAsk,MedianBidP,MedianAskP,Price,AskP0,BidP0,Past_diff1,Past_diff2,Past_diff3,TRAN_P1_ori,diff_3Tick,diff_1Tick
0,9.0,8.0,0.000,-0.470,658.50,658.50,658.12,0.38,0.00,0.00,-0.166250,-0.379870,-0.414818
1,16.0,10.0,2.480,2.350,655.77,655.77,655.11,-2.73,-2.35,0.00,-0.216875,0.034928,-0.116932
2,6.0,26.0,0.770,0.000,655.00,656.00,655.00,-0.77,-3.50,-3.12,0.071250,0.001525,0.152492
3,13.0,6.0,0.000,-1.000,656.00,656.00,655.50,1.00,0.23,-2.50,-0.246250,-0.305344,0.000000
4,10.0,9.0,-0.750,-0.810,656.00,655.01,654.01,0.00,1.00,0.23,-1.303750,-0.454268,-0.150915
5,13.0,12.0,-1.000,-1.010,655.01,655.01,653.01,-0.99,-0.99,0.01,-0.875000,-0.153963,-0.153963
6,14.0,10.0,0.000,-1.485,654.00,654.00,652.01,-1.01,-2.00,-2.00,-0.875000,-0.396941,-0.149616
7,13.0,47.0,0.980,-1.020,653.02,653.98,652.01,-0.98,-1.99,-2.98,-0.788750,-0.155963,0.149847
8,9.0,3.0,-0.020,-0.020,654.00,654.00,653.98,0.98,0.00,-1.01,-0.008750,-0.604882,-0.398150
9,14.0,8.0,2.220,2.580,651.40,653.00,651.20,-2.60,-1.62,-2.60,0.191250,-0.214067,0.091743


In [29]:
# 价格变动能量--PS = mean(P1,P2,P3)
# 交易变化能量--TS = mean(TRAN,t?)
# 买卖变化力度--MS = mean(CountBid * MedianBidP + CountAsk * MedianAskP)
# CountPrice 交易的笔数
# diff_3Tick 未来3Tick价格变化
# diff_1Tick 未来1Tick价格变化
Tick0[['PS','MS','TS','Price','CountPrice','diff_3Tick','diff_1Tick']].head(30)

,PS,MS,TS,Price,CountPrice,diff_3Tick,diff_1Tick
0,0.126667,-3.76,-0.166250,658.50,17.0,-0.379870,-0.414818
1,-1.693333,63.18,-0.216875,655.77,26.0,0.034928,-0.116932
2,-2.463333,4.62,0.071250,655.00,32.0,0.001525,0.152492
3,-0.423333,-6.00,-0.246250,656.00,19.0,-0.305344,0.000000
4,0.410000,-14.79,-1.303750,656.00,19.0,-0.454268,-0.150915
5,-0.656667,-25.12,-0.875000,655.01,25.0,-0.153963,-0.153963
6,-1.670000,-14.85,-0.875000,654.00,24.0,-0.396941,-0.149616
7,-1.983333,-35.20,-0.788750,653.02,60.0,-0.155963,0.149847
8,-0.010000,-0.24,-0.008750,654.00,12.0,-0.604882,-0.398150
9,-2.273333,51.72,0.191250,651.40,22.0,-0.214067,0.091743


In [28]:
check1 = DataSave['diff_3Tick'].nlargest(100).index
DataSave.loc[check1,['PS','MS','TS','CountPrice','diff_3Tick','diff_1Tick']].head(30)

,PS,MS,TS,CountPrice,diff_3Tick,diff_1Tick
1,-0.846667,6.03,0.077500,23.0,0.257903,0.335120
148,0.206667,0.00,0.368750,8.0,0.200603,0.076923
34,-0.833333,4.02,0.451875,9.0,0.190898,0.190898
161,-0.193333,5.90,0.688125,10.0,0.184901,0.108235
139,-0.136667,0.64,-0.336250,9.0,0.151880,0.001504
140,-1.090000,-0.02,0.636875,3.0,0.141141,0.003003
326,0.260000,0.93,1.006875,3.0,0.140683,0.003025
120,-0.823333,31.86,0.138750,27.0,0.129140,0.129140
75,0.146667,-0.87,0.061875,12.0,0.128740,0.061305
223,-0.263333,7.25,0.315625,13.0,0.127017,0.124029


In [6]:
# 在全局Tick上拟合
# inv(X.T * X) * X.T * Y
Tick0['W0'] = 1
check1 = int(Tick0.shape[0] * 0.3)

from numpy import matrix
X  = matrix(Tick0.loc[:check1,['PS','MS','TS','W0']])
Y1 = matrix(Tick0.loc[:check1,'diff_1Tick']).T
Y2 = matrix(Tick0.loc[:check1,'diff_3Tick']).T

W1 = (X.T * X).I * X.T * Y1
W2 = (X.T * X).I * X.T * Y2
print ("fitting done!")
print ("W1:",W1)
print ("W2:",W2)
predict1 = matrix(Tick0.loc[check1:,['PS','MS','TS','W0']]) * W1
predict2 = matrix(Tick0.loc[check1:,['PS','MS','TS','W0']]) * W2
real1 = Tick0.loc[check1:,'diff_1Tick']
real2 = Tick0.loc[check1:,'diff_3Tick']

predicted = pd.DataFrame()
predicted['predict1'] = predict1.T.tolist()[0]
predicted['real1'] = list(real1)
predicted['predict2'] = predict2.T.tolist()[0]
predicted['real2'] = list(real2)

from sklearn.metrics import r2_score
print ("在所有数据中，预测1Tick的r2 score为 ： %.4f 。"%(r2_score(predicted['real1'],predicted['predict1'])))
print ("在所有数据中，预测3Tick的r2 score为 ： %.4f 。"%(r2_score(predicted['real2'],predicted['predict2'])))

fitting done!
W1: [[-0.04036563]
 [ 0.00029536]
 [ 0.06227149]
 [-0.00223492]]
W2: [[-0.02373565]
 [ 0.00074472]
 [ 0.09108552]
 [-0.00286305]]
在所有数据中，预测1Tick的r2 score为 ： 0.2256 。
在所有数据中，预测3Tick的r2 score为 ： 0.2553 。


In [17]:
# 在下单点上拟合
# inv(X.T * X) * X.T * Y
DataSave['W0'] = 1
check2 = int(DataSave.shape[0] * 0.3)

from numpy import matrix
X  = matrix(DataSave.loc[:check2,['PS','MS','TS','W0']])
Y1 = matrix(DataSave.loc[:check2,'diff_1Tick']).T
Y2 = matrix(DataSave.loc[:check2,'diff_3Tick']).T

W1 = (X.T * X).I * X.T * Y1
W2 = (X.T * X).I * X.T * Y2
print ("fitting done!")
print ("W1:",W1)
print ("W2:",W2)
predict1 = matrix(DataSave.loc[check2:,['PS','MS','TS','W0']]) * W1
predict2 = matrix(DataSave.loc[check2:,['PS','MS','TS','W0']]) * W2
real1 = DataSave.loc[check2:,'diff_1Tick']
real2 = DataSave.loc[check2:,'diff_3Tick']

predicted = pd.DataFrame()
predicted['predict1'] = predict1.T.tolist()[0]
predicted['real1'] = list(real1)
predicted['predict2'] = predict2.T.tolist()[0]
predicted['real2'] = list(real2)

from sklearn.metrics import r2_score
print ("在下单数据中，预测1Tick的r2 score为 ： %.4f 。"%(r2_score(predicted['real1'],predicted['predict1'])))
print ("在下单数据中，预测3Tick的r2 score为 ： %.4f 。"%(r2_score(predicted['real2'],predicted['predict2'])),'\n\n')
print (predicted.head(50))

fitting done!
W1: [[-0.06496067]
 [-0.00066342]
 [ 0.08682976]
 [-0.00085151]]
W2: [[-0.01674321]
 [-0.00052708]
 [ 0.09242656]
 [-0.00032338]]
在下单数据中，预测1Tick的r2 score为 ： 0.3069 。
在下单数据中，预测3Tick的r2 score为 ： 0.2853 。 


    predict1     real1  predict2     real2
0  -0.030197 -0.004532 -0.038193 -0.015106
1  -0.028360 -0.003021 -0.036585  0.004532
2  -0.028793  0.034691 -0.042931  0.037707
3  -0.045085 -0.015089 -0.044188  0.003018
4  -0.035921  0.015084 -0.047207  0.015084
5  -0.013049  0.003017  0.008528  0.000000
6  -0.014273 -0.045237 -0.010701  0.009047
7  -0.005863  0.001507  0.000333  0.064809
8  -0.038498  0.000000 -0.016683  0.052744
9  -0.004062 -0.027108  0.011107  0.078313
10  0.023382  0.079819  0.023805  0.111446
11  0.036915  0.073663  0.034444  0.073663
12  0.024918 -0.039087  0.012978 -0.004510
13  0.006311  0.060127 -0.008853  0.060127
14 -0.023657 -0.025564 -0.012699  0.000000
15 -0.039010  0.000000 -0.031443 -0.060130
16 -0.000253 -0.060114  0.003198 -0.031560
17 -0.0

In [11]:
#################################################################################################################################
from sklearn.neighbors import KNeighborsRegressor as KNNR
# diff_3Tick 
KNNRegression = KNNR(n_neighbors = 3,
                     weights = 'distance',#uniform
                     p = 3)
KNNRegression.fit(Tick0.loc[:check1,['PS','MS','TS']],Tick0.loc[:check1,'diff_3Tick'])
predict2_KNN = KNNRegression.predict(Tick0.loc[check1:,['PS','MS','TS']])
print ("在所有数据中，预测3Tick的r2 score为 ： %.4f 。"%(r2_score(Tick0.loc[check1:,'diff_3Tick'],predict2_KNN)))
# diff_1Tick
KNNRegression = KNNR(n_neighbors = 3,
                     weights = 'distance',#uniform
                     p = 3)
KNNRegression.fit(DataSave.loc[:check2,['PS','MS','TS']],DataSave.loc[:check2,'diff_1Tick'])
predict1_KNN = KNNRegression.predict(DataSave.loc[check2:,['PS','MS','TS']])
print ("在下单数据中，预测3Tick的r2 score为 ： %.4f 。"%(r2_score(DataSave.loc[check2:,'diff_3Tick'],predict1_KNN)))

在所有数据中，预测3Tick的r2 score为 ： -0.0062 。
在下单数据中，预测3Tick的r2 score为 ： 0.0869 。
